In [3]:
from collections import defaultdict
from pathlib import Path
import networkx as nx
from tqdm.notebook import tqdm, trange
import pickle

markers = ['o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X']
colors = [
    '#377eb8',
    '#e41a1c',
    '#4daf4a',
    '#984ea3',
    '#ff7f00',
    '#ffff33',
    '#a65628',
    '#f781bf',
    '#999999',
]

graph_names = [
    # social
    'OF',
    'openflights',
    # hypergraphs
    'coauth-DBLP-proj-graph',
    'coauth-MAG-Geology-proj-graph',
    'threads-ask-ubuntu-proj-graph',
    'threads-math-sx-proj-graph',
    'threads-stack-overflow-proj-graph',
    # temporal
    'sx-askubuntu',
    'sx-mathoverflow',
    'sx-stackoverflow',
    'sx-superuser',
]

graph_names_short = [
    # social
    'OF',
    'FL',
    # hypergraphs
    'co-DB',
    'co-GE',
    'th-UB',
    'th-MA',
    'th-SO',
    # temporal
    'sx-UB',
    'sx-MA',
    'sx-SO',
    'sx-SU',
]

name2nameShort = dict(zip(graph_names, graph_names_short))

g2fitting = {
    # social
    'OF': 1,
    'openflights': 2,
    # hypergraphs
    'coauth-DBLP-proj-graph': 3,
    'coauth-MAG-Geology-proj-graph': 3,
    'threads-ask-ubuntu-proj-graph': 1,
    'threads-math-sx-proj-graph': 1,
    'threads-stack-overflow-proj-graph': 1,
    # temporal
    'sx-askubuntu': 2,
    'sx-mathoverflow': 2,
    'sx-stackoverflow': 2,
    'sx-superuser': 2,
}

g2nm = {
    'OF': (897, 71380),
    'openflights': (2905, 15645),

    'coauth-DBLP-proj-graph': (1654109, 7713116),
    'coauth-MAG-Geology-proj-graph': (898648, 4891112),
    'threads-ask-ubuntu-proj-graph': (82075, 182648),
    'threads-math-sx-proj-graph': (152702, 1088735),
    'threads-stack-overflow-proj-graph': (2301070, 20989078),

    'sx-askubuntu': (152599, 453221),
    'sx-mathoverflow': (24668, 187939),
    'sx-stackoverflow': (2572345, 28177464),
    'sx-superuser': (189191, 712870),
}

gt_c_star_wt1 = {
    'OF': 241,
    'openflights': 64,

    'coauth-DBLP-proj-graph': 83,
    'coauth-MAG-Geology-proj-graph': 74,
    'threads-ask-ubuntu-proj-graph': 73,
    'threads-math-sx-proj-graph': 372,
    'threads-stack-overflow-proj-graph': 685,

    'sx-askubuntu': 152,
    'sx-mathoverflow': 185,
    'sx-stackoverflow': 886,
    'sx-superuser': 202,
}

gt_c_star_more = {
    'OF': [(241, 271), (190, 192), (157, 156), (134, 137), (119, 101)],
    'openflights': [(64, 66), (31, 31), (17, 17), (None, None), (None, None)],
    'coauth-DBLP-proj-graph': [(83, 88), (36, 29), (22, 24), (20, 21), (16, 16)],
    'coauth-MAG-Geology-proj-graph': [(74, 92), (52, 49), (34, 40), (28, 30), (24, 21)],
    'threads-ask-ubuntu-proj-graph': [(73, 87), (30, 31), (19, 20), (18, 11), (15, 11)],
    'threads-math-sx-proj-graph': [(372, 401), (145, 153), (114, 114), (84, 67), (63, 59)],
    'threads-stack-overflow-proj-graph': [(685, 750), (208, 205), (134, 129), (97, 82), (74, 72)],
    'sx-askubuntu': [(152, 149), (63, 69), (48, 42), (36, 27), (31, 22)],
    'sx-mathoverflow': [(185, 181), (113, 102), (75, 63), (60, 49), (51, 41)],
    'sx-stackoverflow': [(886, 749), (407, 324), (221, 203), (169, 130), (120, 103)],
    'sx-superuser': [(202, 206), (96, 93), (63, 54), (48, 37), (36, 27)],
}

p_data = Path(f'data')
p_data.mkdir(exist_ok=True)

p_results = Path('results')
p_results.mkdir(exist_ok=True)

Edge = tuple[int, int]
EdgeAndWeight = tuple[int, int, int]
graphs_sorted_m = sorted(graph_names, key=lambda xx: g2nm[xx][1])


def iter_edges(input_graph, with_weight=False, desc='edges'):
    return tqdm(input_graph.edges.data('weight', default=1) if with_weight else input_graph.edges,
                total=input_graph.number_of_edges(), leave=False, desc=desc)


def iter_nodes(input_graph, desc='nodes'):
    return tqdm(input_graph.nodes, total=input_graph.number_of_nodes(), leave=False, desc=desc)


def data_exist(ds, data_name_, layer_index=None):
    if layer_index is not None:
        return (p_data / data_name_ / f'{ds}.{data_name_}_layer{layer_index}').is_file()
    return (p_data / data_name_ / f'{ds}.{data_name_}').is_file()


def data_file_path(ds, data_name_, layer_index=None, write=False):
    if write:
        (p_data / data_name_).mkdir(exist_ok=True)
        mode = 'wb'
    else:
        mode = 'rb'
    if layer_index is not None:
        return p_data / data_name_ / f'{ds}.{data_name_}_layer{layer_index}', mode
    return p_data / data_name_ / f'{ds}.{data_name_}', mode


def save_data(data, ds, data_name_, layer_index=None):
    with open(*data_file_path(ds, data_name_, write=True, layer_index=layer_index)) as f_:
        pickle.dump(data, f_)


def load_data(ds, data_name_, layer_index=None):
    with open(*data_file_path(ds, data_name_, write=False, layer_index=layer_index)) as f_:
        return pickle.load(f_)


def min_max_tuple(xx, yy):
    return min(xx, yy), max(xx, yy)


def reorder_nodes(input_graph):
    return nx.convert_node_labels_to_integers(input_graph)


def take_gcc(input_graph):
    return input_graph.subgraph(max(nx.connected_components(input_graph), key=len))

p = dict()

In [16]:
import pickle
import networkx as nx
from pathlib import Path

# read the raw data and convert it into graphs
# saving edge text file, edges, weights, and edges-and-weights lists
p['raw_data'] = Path('raw_data')
assert p['raw_data'].is_dir()
data_names = [
    'graph',
    'edge_txt',
    'edges',
    'weights',
    'edges_and_weights',
    'neighbors_list',
    'number_of_CNs_list',
]
for data_name in data_names:
    p[data_name] = p_data / data_name
    p[data_name].mkdir(exist_ok=True)

for f_raw_data in p['raw_data'].iterdir():
    graph_name = [name_ for name_ in graph_names if name_ in f_raw_data.name][0]
    G = nx.Graph()
    with f_raw_data.open() as f:
        dd = f.readlines()
    for d in dd:
        u, v, w = map(int, d.split())
        G.add_edge(u, v, weight=w)
    G = take_gcc(G)
    G = reorder_nodes(G)
    save_data(G, graph_name, 'graph')
    print(graph_name, G)
    n, m = g2nm[graph_name]
    assert (n, m) == (G.number_of_nodes(), G.number_of_edges())
    neighbors_list = [set(G[v]) for v in range(n)]
    edges = []
    weights = []
    edges_and_weights = []
    number_of_CNs_list = []
    with (p['edge_txt'] / f'{graph_name}.edge_txt').open('w+') as f:
        for u, v, w in iter_edges(G, with_weight=True, desc=graph_name):
            u, v = min_max_tuple(u, v)
            edges.append((u, v))
            weights.append(w)
            edges_and_weights.append((u, v, w))
            cn_uv = len(neighbors_list[u] & neighbors_list[v])
            number_of_CNs_list.append(cn_uv)
            f.write(f'{u} {v} {w}\n')
    save_data(neighbors_list, graph_name, 'neighbors_list')
    save_data(edges, graph_name, 'edges')
    save_data(weights, graph_name, 'weights')
    save_data(edges_and_weights, graph_name, 'edges_and_weights')
    save_data(number_of_CNs_list, graph_name, 'number_of_CNs_list')

coauth-DBLP-proj-graph Graph with 1654109 nodes and 7713116 edges


coauth-DBLP-proj-graph:   0%|          | 0/7713116 [00:00<?, ?it/s]

sx-superuser Graph with 189191 nodes and 712870 edges


sx-superuser:   0%|          | 0/712870 [00:00<?, ?it/s]

threads-math-sx-proj-graph Graph with 152702 nodes and 1088735 edges


...

sx-mathoverflow:   0%|          | 0/187939 [00:00<?, ?it/s]

sx-stackoverflow Graph with 2572345 nodes and 28177464 edges


sx-stackoverflow:   0%|          | 0/28177464 [00:00<?, ?it/s]

threads-ask-ubuntu-proj-graph Graph with 82075 nodes and 182648 edges


threads-ask-ubuntu-proj-graph:   0%|          | 0/182648 [00:00<?, ?it/s]

In [ ]:
import subprocess

# number of common neighbors c -> number of pairs sharing c CNs
# for each dataset, among all the node pairs, for each number c of common neighbors,
# we compute how many pairs share c CNs
# we use the cpp program cn_pairs.cpp

cmd_compile = ['g++', '-O3', '-std=c++2a', 'cn_pairs.cpp', '-o', 'cn_pairs', '-lpthread']
subprocess.run(cmd_compile)
for graph_name in graphs_sorted_m:
    print(graph_name)
    cmd_run = ['./cn_pairs', name2nameShort[graph_name]]
    subprocess.run(cmd_run)
    # string outfile = "data/numberOfCN2numberOfPairs_cpp/" + dataset_full + ".txt";
    with open(p_data / f'numberOfCN2numberOfPairs_cpp/{graph_name}.txt') as f:
        dd = f.readlines()
    cn2p = dict()
    for d in dd:
        try:
            cn, p = map(int, d.split())
            cn2p[cn] = p
        except:
            continue
    save_data(cn2p, graph_name, f'numberOfCN2numberOfPairs')


In [19]:
# make the layers
data_names = [
    'layers'
]
for data_name in data_names:
    p[data_name] = p_data / data_name
    p[data_name].mkdir(exist_ok=True)

for graph_name in graphs_sorted_m:
    G = load_data(graph_name, 'graph')
    n, m = g2nm[graph_name]
    weights = load_data(graph_name, 'weights')
    edges_and_weights = load_data(graph_name, 'edges_and_weights')
    w_max = max(weights)
    for i_layer in trange(2, min(w_max, 10) + 1):
        edges_and_weights = [(u, v, w) for u, v, w in edges_and_weights if w >= i_layer]
        G_i = nx.Graph()
        for u, v, w in edges_and_weights:
            G_i.add_edge(u, v, weight=w)
        save_data(G_i, graph_name, 'layers', layer_index=i_layer)
        print(graph_name, f'layer{i_layer}', G_i)


  0%|          | 0/9 [00:00<?, ?it/s]

openflights layer2 Graph with 1351 nodes and 4608 edges
openflights layer3 Graph with 624 nodes and 1507 edges
openflights layer4 Graph with 319 nodes and 564 edges
openflights layer5 Graph with 152 nodes and 210 edges
openflights layer6 Graph with 76 nodes and 80 edges
openflights layer7 Graph with 33 nodes and 28 edges
openflights layer8 Graph with 11 nodes and 6 edges
openflights layer9 Graph with 6 nodes and 3 edges
openflights layer10 Graph with 4 nodes and 2 edges


  0%|          | 0/9 [00:00<?, ?it/s]

OF layer2 Graph with 856 nodes and 47266 edges
OF layer3 Graph with 816 nodes and 35456 edges
OF layer4 Graph with 782 nodes and 28546 edges
OF layer5 Graph with 761 nodes and 23471 edges
OF layer6 Graph with 747 nodes and 20008 edges
OF layer7 Graph with 728 nodes and 17472 edges
OF layer8 Graph with 714 nodes and 15349 edges
OF layer9 Graph with 706 nodes and 13666 edges
OF layer10 Graph with 684 nodes and 12272 edges


  0%|          | 0/9 [00:00<?, ?it/s]

...

coauth-DBLP-proj-graph layer2 Graph with 750120 nodes and 2269679 edges
coauth-DBLP-proj-graph layer3 Graph with 465015 nodes and 1085489 edges
coauth-DBLP-proj-graph layer4 Graph with 330533 nodes and 654182 edges
coauth-DBLP-proj-graph layer5 Graph with 250618 nodes and 437964 edges
coauth-DBLP-proj-graph layer6 Graph with 197325 nodes and 312595 edges
coauth-DBLP-proj-graph layer7 Graph with 159509 nodes and 233294 edges
coauth-DBLP-proj-graph layer8 Graph with 131523 nodes and 179278 edges
coauth-DBLP-proj-graph layer9 Graph with 110096 nodes and 141170 edges
coauth-DBLP-proj-graph layer10 Graph with 93133 nodes and 113328 edges


  0%|          | 0/9 [00:00<?, ?it/s]

threads-stack-overflow-proj-graph layer2 Graph with 286340 nodes and 1168210 edges
threads-stack-overflow-proj-graph layer3 Graph with 95029 nodes and 350871 edges
threads-stack-overflow-proj-graph layer4 Graph with 48314 nodes and 170618 edges
threads-stack-overflow-proj-graph layer5 Graph with 30050 nodes and 102334 edges
threads-stack-overflow-proj-graph layer6 Graph with 20891 nodes and 68997 edges
threads-stack-overflow-proj-graph layer7 Graph with 15559 nodes and 49789 edges
threads-stack-overflow-proj-graph layer8 Graph with 12103 nodes and 37685 edges
threads-stack-overflow-proj-graph layer9 Graph with 9837 nodes and 29614 edges
threads-stack-overflow-proj-graph layer10 Graph with 8202 nodes and 24044 edges


  0%|          | 0/9 [00:00<?, ?it/s]

sx-stackoverflow layer2 Graph with 1691246 nodes and 9871784 edges
sx-stackoverflow layer3 Graph with 1074137 nodes and 4137454 edges
sx-stackoverflow layer4 Graph with 711977 nodes and 2055034 edges
sx-stackoverflow layer5 Graph with 480249 nodes and 1126093 edges
sx-stackoverflow layer6 Graph with 330553 nodes and 666010 edges
sx-stackoverflow layer7 Graph with 232121 nodes and 420890 edges
sx-stackoverflow layer8 Graph with 166821 nodes and 281717 edges
sx-stackoverflow layer9 Graph with 122713 nodes and 197844 edges
sx-stackoverflow layer10 Graph with 92370 nodes and 144587 edges


In [ ]:
import subprocess

# number of common neighbors c -> number of pairs sharing c CNs for the layers
# we use the cpp program cn_pairs_layers.cpp

# string edge_input = "data/edge_txt_layers/" + dataset_full + "_layer" + to_string(i_layer) + ".edge_txt";
data_names = [
    'edge_txt_layers'
]
for data_name in data_names:
    p[data_name] = p_data / data_name
    p[data_name].mkdir(exist_ok=True)

cmd_compile = ['g++', '-O3', '-std=c++2a', 'cn_pairs_layers.cpp', '-o', 'cn_pairs_layers', '-lpthread']
subprocess.run(cmd_compile)

for graph_name in graphs_sorted_m:
    print(graph_name)
    n, m = g2nm[graph_name]
    weights = load_data(graph_name, 'weights')
    w_max = max(weights)
    for i_layer in trange(2, min(w_max, 5) + 1):
        G_i = load_data(graph_name, 'layers', layer_index=i_layer)
        print(graph_name, f'layer{i_layer}', G_i)
        G_i = reorder_nodes(G_i)
        n_i, m_i = G_i.number_of_nodes(), G_i.number_of_edges()
        with open(p['edge_txt_layers'] / f'{graph_name}_layer{i_layer}.edge_txt', 'w+') as f:
            f.write(f'{n} {m}\n')
            for u, v in iter_edges(G_i):
                f.write(f'{u} {v}\n')
            cmd_run = ['./cn_pairs_layers', name2nameShort[graph_name], str(i_layer)]
            subprocess.run(cmd_run)
        with open(p_data / f'numberOfCN2numberOfPairs_cpp/{graph_name}_layer{i_layer}.txt') as f:
            dd = f.readlines()
        cn2p = dict()
        for d in dd:
            try:
                cn, p = map(int, d.split())
                cn2p[cn] = p
            except:
                continue
        save_data(cn2p, graph_name, f'numberOfCN2numberOfPairs_layers', layer_index=i_layer)

In [20]:
import math
from itertools import product

# metrics
metrics = [
    'CN',
    'SA',
    'JC',
    'HP',
    'HD',
    'SI',
    'LI',
    'AA',
    'RA',
    'PA',
    'FM',
    'DL',
]
data_names = [
    f'{metric}-list' for metric in metrics
]
for data_name in data_names:
    p[data_name] = p_data / data_name
    p[data_name].mkdir(exist_ok=True)

for graph_name in graphs_sorted_m:
    edges = load_data(graph_name, 'edges')
    neighbors_list = load_data(graph_name, 'neighbors_list')
    metric2list = defaultdict(list)
    degrees = [len(N) for N in neighbors_list]
    for u, v in tqdm(edges, desc=graph_name, leave=False):
        Nu, Nv = neighbors_list[u], neighbors_list[v]
        du, dv = len(Nu), len(Nv)
        metric2value = defaultdict(float)
        CN_uv = Nu & Nv
        cn_uv = len(CN_uv)
        metric2value['CN'] = cn_uv
        metric2value['SA'] = cn_uv / math.sqrt(du * dv)
        metric2value['JC'] = cn_uv / (du + dv - cn_uv)
        metric2value['HP'] = cn_uv / min(du, dv)
        metric2value['HD'] = cn_uv / max(du, dv)
        metric2value['SI'] = cn_uv / (du + dv)
        metric2value['LI'] = cn_uv / (du * dv)
        for x in CN_uv:
            metric2value['AA'] += 1 / math.log(degrees[x])
            metric2value['RA'] += 1 / degrees[x]
        metric2value['PA'] = du * dv
        metric2value['FM'] = cn_uv
        for x, y in product(Nu - Nv, Nv - Nu):
            if y in neighbors_list[x]:
                metric2value['FM'] += 1
        metric2value['DL'] = du + dv - 2
        for metric in metrics:
            metric2list[metric].append(metric2value[metric])
    for metric, metric_list in metric2list.items():
        save_data(metric_list, graph_name, f'{metric}-list')

openflights:   0%|          | 0/15645 [00:00<?, ?it/s]

OF:   0%|          | 0/71380 [00:00<?, ?it/s]

threads-ask-ubuntu-proj-graph:   0%|          | 0/182648 [00:00<?, ?it/s]

sx-mathoverflow:   0%|          | 0/187939 [00:00<?, ?it/s]

sx-askubuntu:   0%|          | 0/453221 [00:00<?, ?it/s]

sx-superuser:   0%|          | 0/712870 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import subprocess

# metric (using cpp)
# you may run the cpp version of the above cell (metrics.cpp) for higher speed
# this cell will run the cpp program and read the output

# you can change the variable "graph_names_large"
# for changing the range of the datasets
# on which the cpp program runs
metrics = [
    'CN',
    'SA',
    'JC',
    'HP',
    'HD',
    'SI',
    'LI',
    'AA',
    'RA',
    'PA',
    'FM',
    'DL',
]
data_names = [
    f'{metric}-list' for metric in metrics
]
for data_name in data_names:
    p[data_name] = p_data / data_name
    p[data_name].mkdir(exist_ok=True)

graph_names_large = graphs_sorted_m[-6:]
# compile the cpp file
cmd_compile = ['g++', '-O3', '-std=c++2a', 'metrics.cpp', '-o', 'metric', '-lpthread']
subprocess.run(cmd_compile)
for graph_name in graph_names_large:
    print(graph_name)
    cmd_run = ['./metric', name2nameShort[graph_name]]
    subprocess.run(cmd_run)
    for metric in metrics:
        metric_list = []
        with open(p_data / f'metrics_cpp/{graph_name}_{metric.lower()}.txt') as f:
            dd = f.readlines()
        for d in dd:
            try:
                data_float = float(d)
                metric_list.append(data_float)
            except:
                continue
        save_data(metric_list, graph_name, f'{metric}-list')

182647/182648sx-mathoverflow
187938/187939sx-askubuntu
453220/453221sx-superuser
712869/712870threads-math-sx-proj-graph
395303/1088735

In [ ]:
from sknetwork.topology import CoreDecomposition

# edge coreness

metric_names = [
    'coreness_list',
    'edge_coreness_list',
]

for graph_name in graphs_sorted_m:
    n, m = g2nm[graph_name]
    G = load_data(graph_name, 'graph')
    adjacency = nx.adjacency_matrix(G, weight=None)
    core = CoreDecomposition()
    labels = core.fit_transform(adjacency)
    coreness_list = [0 for _ in range(n)]
    for i, v in enumerate(iter_nodes(G)):
        coreness_list[v] = labels[i]
    save_data(coreness_list, graph_name, 'coreness_list')
    edges = load_data(graph_name, 'edges')
    edge_coreness_list = []
    for u, v in edges:
        cu, cv = coreness_list[u], coreness_list[v]
        edge_coreness_list.append(min(cu, cv))
    save_data(edge_coreness_list, graph_name, 'edge_coreness_list')

In [ ]:
import os
from itertools import product

# local path

if '9-edge_weight' not in str(Path.cwd()):
    os.chdir('9-edge_weight')

metrics = [
    'LP',
]
data_names = [
    f'{metric}-list' for metric in metrics
]
for data_name in data_names:
    p[data_name] = p_data / data_name
    p[data_name].mkdir(exist_ok=True)

for graph_name in graphs_sorted_m:
    n, m = g2nm[graph_name]
    G = load_data(graph_name, 'graph')
    edges = load_data(graph_name, 'edges')
    number_of_CNs_list = load_data(graph_name, 'number_of_CNs_list')
    neighbors_list = load_data(graph_name, 'neighbors_list')
    CN_list = load_data(graph_name, 'CN_list')
    LP_list = []
    epsilon = 0.001
    for i, (u, v) in enumerate(tqdm(edges)):
        LP_uv = 1. + CN_list[i]
        Nu, Nv = neighbors_list[u], neighbors_list[v]
        for x, y in product(Nu, Nv):
            if x == y:
                continue
            if y in neighbors_list[x]:
                LP_uv += epsilon
        LP_list.append(LP_uv)
    save_data(LP_list, graph_name, 'LP_list')


In [ ]:
import subprocess

# local path (using cpp)
# you may run the cpp version of the above cell (local_path.cpp) for higher speed
# this cell will run the cpp program and read the output

# you can change the variable "graph_names_large"
# for changing the range of the datasets
# on which the cpp program runs


metrics = [
    'LP',
]
data_names = [
    f'{metric}-list' for metric in metrics
]
for data_name in data_names:
    p[data_name] = p_data / data_name
    p[data_name].mkdir(exist_ok=True)

graph_names_large = graphs_sorted_m
# compile the cpp file
cmd_compile = ['g++', '-O3', '-std=c++2a', 'local_path.cpp', '-o', 'local_path', '-lpthread']
subprocess.run(cmd_compile)
for graph_name in graph_names_large:
    print(graph_name)
    cmd_run = ['./local_path', name2nameShort[graph_name]]
    subprocess.run(cmd_run)
    for metric in metrics:
        metric_list = []
        with open(p_data / f'metric_cpp/{graph_name}_{metric.lower()}.txt') as f:
            dd = f.readlines()
        for d in dd:
            try:
                data_float = float(d)
                metric_list.append(data_float)
            except:
                continue
        save_data(metric_list, graph_name, f'{metric}-list')


In [ ]:
import os
import pickle

# edge betweenness

if '9-edge_weight' not in str(Path.cwd()):
    os.chdir('9-edge_weight')

metric_names = [
    'EB',  # edge betweenness
]
for data_name in data_names:
    p[data_name] = p_data / data_name
    p[data_name].mkdir(exist_ok=True)

for graph_name in graphs_sorted_m:
    print(graph_name)
    n, m = g2nm[graph_name]
    G = load_data(graph_name, 'graph')
    edges = load_data(graph_name, 'edges')
    e2btwness = nx.edge_betweenness_centrality(G)
    EB_list = []
    for i, e in enumerate(tqdm(edges)):
        u, v = e
        try:
            EB_list.append(e2btwness[e])
        except:
            EB_list.append(e2btwness[(v, u)])
    save_data(EB_list, graph_name, 'EB-list')

In [ ]:
import subprocess

# edge betweenness (using cpp)
# you may run the cpp version of the above cell (eb.cpp) for higher speed
# this cell will run the cpp program and read the output

# you can change the variable "graph_names_large"
# for changing the range of the datasets
# on which the cpp program runs

metrics = [
    'EB',
]
data_names = [f'{metric}-list' for metric in metrics]
data_names += [
    'edge_txt_BGL',
]
for data_name in data_names:
    p[data_name] = p_data / data_name
    p[data_name].mkdir(exist_ok=True)

graph_names_large = graphs_sorted_m
# compile the cpp file
cmd_compile = ['g++', '-O3', '-std=c++2a', 'eb.cpp', '-o', 'eb', '-lpthread']
subprocess.run(cmd_compile)
for graph_name in graph_names_large:
    print(graph_name)
    n, m = g2nm[graph_name]
    edges = load_data(graph_name, 'edges')
    with open(p['edge_txt_BGL'] / f'{graph_name}.edge_txt_BGL', 'w+') as f:
        f.write(f'{2 * m}\n')
        for u, v in edges:
            f.write(f'{u} - {v}\n')
            f.write(f'{v} - {u}\n')
    cmd_run = ['./eb', name2nameShort[graph_name]]
    subprocess.run(cmd_run)
    norm_term = n * (n - 1) / 2
    with open(p_data / f'metric_cpp/{graph_name}_eb.txt') as f:
        dd = f.readlines()
    e2eb = dict()
    for d in dd:
        u, v, eb = d.split()
        u, v, eb = int(u), int(v), float(eb)
        e2eb[min_max_tuple(u, v)] = eb
    edges = load_data(graph_name, 'edges')
    EB_list = []
    for i, e in enumerate(tqdm(edges)):
        u, v = e
        try:
            EB_list.append(e2eb[e] / norm_term)
        except:
            EB_list.append(e2eb[(v, u)] / norm_term)
    save_data(EB_list, graph_name, 'EB_list')
